<a href="https://colab.research.google.com/github/mhdykz/processing-techniques-to-denoise-images-corrupted-by-salt-and-pepper-noise/blob/main/classic_image_processing_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def remove_strong_noise(image):
    noisy_image = image.copy()
    noisy_image[noisy_image == 0] = 1
    noisy_image[noisy_image == 255] = 254
    return noisy_image

In [3]:
def label_objects(image):
    num_labels, labels = cv2.connectedComponents(image)
    return num_labels, labels


In [4]:
def add_salt_and_pepper_noise(image, noise_density):
    noisy_image = image.copy()
    total_pixels = image.size
    num_noise_pixels = int(noise_density * total_pixels)

    salt_coords = [np.random.randint(0, i - 1, num_noise_pixels) for i in image.shape]
    noisy_image[salt_coords[0], salt_coords[1]] = 255

    pepper_coords = [np.random.randint(0, i - 1, num_noise_pixels) for i in image.shape]
    noisy_image[pepper_coords[0], pepper_coords[1]] = 0

    return noisy_image


In [5]:
def find_noise_pixels(image, labels):
    noise_pixels = np.where((image == 0) | (image == 255))
    noise_positions = list(zip(noise_pixels[0], noise_pixels[1]))

    noise_with_labels = [(pos, labels[pos[0], pos[1]]) for pos in noise_positions]
    return noise_with_labels


In [6]:
def generate_possible_matrices(image, noise_position, label, labels):
    x, y = noise_position
    object_pixels = np.where(labels == label)

    max_size = 5
    possible_matrices = []

    for size in range(3, max_size+1, 2):
        half_size = size // 2
        start_x = max(0, x - half_size)
        end_x = min(image.shape[0], x + half_size + 1)
        start_y = max(0, y - half_size)
        end_y = min(image.shape[1], y + half_size + 1)

        matrix = image[start_x:end_x, start_y:end_y]
        possible_matrices.append(matrix)

    return possible_matrices

In [7]:
def minimize_variance(possible_matrices):
    min_variance = float('inf')
    best_value = None

    for matrix in possible_matrices:
        variance = np.var(matrix)
        if variance < min_variance:
            min_variance = variance
            best_value = np.median(matrix)

    return best_value

In [8]:
def apply_kazi_filter(image, labels):
    image = remove_strong_noise(image)

    noise_with_labels = find_noise_pixels(image, labels)

    for noise_position, label in noise_with_labels:
        possible_matrices = generate_possible_matrices(image, noise_position, label, labels)
        best_value = minimize_variance(possible_matrices)

        x, y = noise_position
        image[x, y] = best_value

    return image